In [1]:
import numpy as np 
import pandas as pd 
% matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns

In [44]:
data = pd.read_csv('amazon_baby.csv')

# Lets explore the data

In [45]:
data.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [33]:
data.describe()

,rating
count,183531.000000
mean,4.120448
std,1.285017
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [46]:
data.isnull().sum()

name      318
review    829
rating      0
dtype: int64

In [48]:
data = data[data.review.notnull()]
data.isnull().sum()

name      318
review      0
rating      0
dtype: int64

# Build the word count vector for each review

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\اورجينال\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [49]:
text = data['review']
#data['word counter'] = nltk.word_tokenize(text)
#data.head()


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
data['word counter'] = vectorizer.fit(text)
data.head()

,name,review,rating,word counter
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,"CountVectorizer(analyzer='word', binary=False,..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,"CountVectorizer(analyzer='word', binary=False,..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,"CountVectorizer(analyzer='word', binary=False,..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,"CountVectorizer(analyzer='word', binary=False,..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,"CountVectorizer(analyzer='word', binary=False,..."


In [ ]:
text_2 = 'hghjg hjhssu jvd hyi j koooko '
